# Waveform and spectrogram display

Select an audio file from the dropdown list to display its waveform and spectrogram.

In [ ]:
import os
import parselmouth
import numpy as np

from phonlab.utils import dir2df
from bokeh_phon.utils import remote_jupyter_proxy_url_callback, default_jupyter_url
from bokeh_phon.models.audio_button import AudioButton

from bokeh.plotting import figure
from bokeh.models import BoxAnnotation, BoxSelectTool, BoxZoomTool, ColumnDataSource, \
    CrosshairTool, LogColorMapper, Select, Slider, ZoomInTool, ZoomOutTool
from bokeh.io import show, output_notebook, push_notebook
from bokeh.layouts import column, gridplot
from bokeh.events import SelectionGeometry
from bokeh.palettes import Greys256
r_Greys256 = list(reversed(Greys256))

# The remote_jupyter_proxy_url function is required when running on a BinderHub instance.
# Change the default_jupyter_url value to match the hostname of your instance after it has
# started. The current value is the most frequent result when launching from mybinder.org.
# Note that default_jupyter_url must be imported from bokeh_phon.utils in order for it to be
# available to the remote_jupyter_proxy_url function.

# Change to None if running locally.
#default_jupyter_url = None

output_notebook()

In [ ]:
params = {
    'low_thresh_power': 13
}
def myapp(doc):
    def load_wav_cb(attr, old, new):
        if not new.endswith('.wav'):
            return
        snd = parselmouth.Sound(new)
        if snd.n_channels > 1:
            snd = snd.convert_to_mono()
        playvisbtn.channels = channels
        playvisbtn.visible = True
        playselbtn.channels = channels
        playselbtn.visible = True
        sgrams[0] = snd.to_spectrogram()
        spec0img.glyph.dw = sgrams[0].x_grid().max()
        spec0img.glyph.dh = sgrams[0].y_grid().max()
        playvisbtn.fs = snd.sampling_frequency
        playvisbtn.start = snd.start_time
        playvisbtn.end = snd.end_time
        playselbtn.fs = snd.sampling_frequency
        playselbtn.start = 0.0
        playselbtn.end = 0.0
        n_chan = snd.values.shape[0]
        source.data = dict(
            seconds=snd.ts().astype(np.float32),
            ch0=snd.values[0,:].astype(np.float32),
        )
        ch0.visible = True
        spec0cmap.low = _low_thresh()
        specsource.data = dict(
            sgram0=[sgrams[0].values.astype(np.float32)]
        )
        spec0.visible = True

    def x_range_cb(attr, old, new):
        if attr == 'start':
            playvisbtn.start = new
        elif attr == 'end':
            playvisbtn.end = new

    def selection_cb(e):
        '''Handle data range selection event.'''
        playselbtn.start = e.geometry['x0']
        playselbtn.end = e.geometry['x1']
        selbox.left = e.geometry['x0']
        selbox.right = e.geometry['x1']
        selbox.visible = True

    def low_thresh_cb(attr, old, new):
        params['low_thresh_power'] = new
        spec0cmap.low = _low_thresh()

    def _low_thresh():
        return sgrams[0].values.min() \
               + sgrams[0].values.std()**params['low_thresh_power']

    datadir = '../resource'
    fdf = dir2df(datadir, fnpat='.*\.wav$')
    fdf['fpath'] = [
        os.path.normpath(
            os.path.join(datadir, relname)
        ) for relname in fdf.relpath.str.cat(fdf.fname, sep='/')
    ]
    options = [('', 'Choose an audio file to display')]
    options.extend(
        list(fdf.loc[:,['fpath', 'fname']].itertuples(index=False, name=None))
    )
    fselect = Select(options=options, value='')
    fselect.on_change('value', load_wav_cb)
    source = ColumnDataSource(data=dict(seconds=[], ch0=[]))
    channels = ['ch0']

    playvisbtn = AudioButton(
        label='Play visible signal', source=source, channels=channels,
        visible=False
    )
    playselbtn = AudioButton(
        label='Play selected signal', source=source, channels=channels,
        visible=False
    )
    
    # Instantiate and share specific select/zoom tools so that
    # highlighting is synchronized on all plots.
    boxsel = BoxSelectTool(dimensions='width')
    boxzoom = BoxZoomTool(dimensions='width')
    zoomin = ZoomInTool(dimensions='width')
    zoomout = ZoomOutTool(dimensions='width')
    crosshair = CrosshairTool(dimensions='height')
    shared_tools = [
        'xpan', boxzoom, boxsel, crosshair, 'undo', 'redo',
        zoomin, zoomout, 'save', 'reset'
    ]

    figargs = dict(
        tools=shared_tools,
    )
    ch0 = figure(name='ch0', tooltips=[("time", "$x{0.0000}")], **figargs)
    ch0.line(x='seconds', y='ch0', source=source, nonselection_line_alpha=0.6)
    # Link pan, zoom events for plots with x_range.
    ch0.x_range.on_change('start', x_range_cb)
    ch0.x_range.on_change('end', x_range_cb)
    ch0.on_event(SelectionGeometry, selection_cb)
    low_thresh = 0.0
    sgrams = [np.ones((1, 1))]
    specsource = ColumnDataSource(data=dict(sgram0=[sgrams[0]]))
    spec0 = figure(
        name='spec0',
        x_range=ch0.x_range, # Keep times synchronized
        tooltips=[("time", "$x{0.0000}"), ("freq", "$y{0.0000}"), ("value", "@sgram0{0.000000}")],
        **figargs
    )
    spec0.x_range.range_padding = spec0.y_range.range_padding = 0
    spec0cmap = LogColorMapper(palette=r_Greys256)
    low_thresh_slider = Slider(
        start=5.0, end=25.0, step=0.25, value=params['low_thresh_power'], title=None
    )
    spec0img = spec0.image(
        image='sgram0',
        x=0, y=0,
        color_mapper=spec0cmap,
        level='image',
        source=specsource
    )
    spec0.grid.grid_line_width = 0.0
    low_thresh_slider.on_change('value', low_thresh_cb)
    selbox = BoxAnnotation(
        name='selbox',
        left=0.0, right=0.0,
        fill_color='green', fill_alpha=0.1,
        line_color='green', line_width=1.5, line_dash='dashed',
        visible=False
    )
    ch0.add_layout(selbox)
    spec0.add_layout(selbox)
    spec0.on_event(SelectionGeometry, selection_cb)
    grid = gridplot(
        [ch0, spec0],
        ncols=1,
        plot_height=200,
        toolbar_location='left',
        toolbar_options={'logo': None},
        merge_tools=True
    )
    mainLayout = column(
        fselect, playvisbtn, playselbtn, grid, low_thresh_slider,
        name='mainLayout'
    )
    doc.add_root(mainLayout)
    return doc

# The notebook_url parameter is required when running in a BinderHub instance.
# If running a local notebook, omit that parameter.
if default_jupyter_url is None:
    show(myapp)    # For running a local notebook
else:
    show(myapp, notebook_url=remote_jupyter_proxy_url_callback)